# Initial codes

This notebook illustrates how Bayesian inference can be used to infer response rates of each group (basket) in a Basket trial.

In [2]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [6]:
import sys, os
from os.path import exists

sys.path.append('..')
sys.path.append('.')

In [7]:
import numpy as np
import pandas as pd
import arviz as az

In [10]:
from pyBasket.common import load_obj

In [11]:
def get_output_filenames(result_dir, scenario, with_clustering_info, n_clusters):
    base_filename = f'scenario_{scenario}_clustering_{with_clustering_info}'
    if with_clustering_info:
        base_filename += f'_ncluster_{n_clusters}'

    out_trial_results = os.path.join(result_dir, base_filename + '_trial_results.p')
    return out_trial_results

In [8]:
out_dir = os.path.abspath(os.path.join('..', 'scripts', 'results'))
out_dir

'/Users/joewandy/Work/git/pyBasket/scripts/results'

In [18]:
scenarios = range(0, 6)
clustering = [False, True]
n_clusters = [5, 10]

In [23]:
data = {}

for scenario in scenarios:
    for cl in clustering:
        if cl:
            for nc in n_clusters:
                fname = get_output_filenames(out_dir, scenario, cl, nc)
                print(fname)                
                trial_results = load_obj(fname)
                key = (scenario, cl, nc)
                data[key] = trial_results
        else:
            nc = None
            fname = get_output_filenames(out_dir, scenario, cl, nc)
            print(fname)
            tr = load_obj(fname)
            trial_results = load_obj(fname)
            key = (scenario, cl, nc)
            data[key] = trial_results

/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_10_trial_results.p


In [24]:
scenario = 0
cl = False
nc = None
key = (scenario, cl, nc)
print(key)
trial_result = data[key]
trial_result

(0, False, None)


In [27]:
for analysis_name in trial_result[0].idfs:
    print(analysis_name)
    idfs = trial_result[0].idfs[analysis_name]
    for df in idfs:
        display(df)

independent


,prob,Q,effective,group_status,group_nnz,group_total
k,,,,,,
0,0.15,0.05,True,OPEN,3,14
1,0.07,0.05,True,OPEN,2,14
2,0.73,0.05,True,OPEN,6,14
3,0.30,0.05,True,OPEN,3,14
4,0.26,0.05,True,OPEN,3,14
5,0.03,0.05,False,EARLY_STOP_FUTILE,1,14


,prob,Q,effective,group_status,group_nnz,group_total
k,,,,,,
0,0.79,0.888,False,COMPLETED_INEFFECTIVE,6,24
1,0.57,0.888,False,COMPLETED_INEFFECTIVE,5,24
2,1.00,0.888,True,COMPLETED_EFFECTIVE,11,24
3,0.40,0.888,False,COMPLETED_INEFFECTIVE,4,24
4,0.59,0.888,False,COMPLETED_INEFFECTIVE,5,24
5,0.16,0.888,False,EARLY_STOP_FUTILE,1,14


independent_bern


,prob,Q,effective,group_status,group_nnz,group_total
k,,,,,,
0,0.76,0.05,True,OPEN,3,14
1,0.66,0.05,True,OPEN,2,14
2,0.97,0.05,True,OPEN,6,14
3,0.81,0.05,True,OPEN,3,14
4,0.76,0.05,True,OPEN,3,14
5,0.52,0.05,True,OPEN,1,14


...

,prob,Q,effective,group_status,group_nnz,group_total
k,,,,,,
0,0.02,0.05,False,EARLY_STOP_FUTILE,3,14
1,0.00,0.05,False,EARLY_STOP_FUTILE,2,14
2,0.08,0.05,True,OPEN,6,14
3,0.05,0.05,True,OPEN,3,14
4,0.02,0.05,False,EARLY_STOP_FUTILE,3,14
5,0.00,0.05,False,EARLY_STOP_FUTILE,1,14


,prob,Q,effective,group_status,group_nnz,group_total
k,,,,,,
0,0.70,0.496,True,EARLY_STOP_FUTILE,3,14
1,0.61,0.496,True,EARLY_STOP_FUTILE,2,14
2,0.90,0.496,True,COMPLETED_EFFECTIVE,11,24
3,0.58,0.496,True,COMPLETED_EFFECTIVE,4,24
4,0.70,0.496,True,EARLY_STOP_FUTILE,3,14
5,0.48,0.496,False,EARLY_STOP_FUTILE,1,14


pyBasket


,prob,Q,effective,group_status,group_nnz,group_total
k,,,,,,
0,0.83,0.05,True,OPEN,3,14
1,0.73,0.05,True,OPEN,2,14
2,0.96,0.05,True,OPEN,6,14
3,0.86,0.05,True,OPEN,3,14
4,0.82,0.05,True,OPEN,3,14
5,0.37,0.05,True,OPEN,1,14


,prob,Q,effective,group_status,group_nnz,group_total
k,,,,,,
0,0.99,0.993,False,COMPLETED_INEFFECTIVE,6,24
1,0.94,0.993,False,COMPLETED_INEFFECTIVE,5,24
2,1.00,0.993,True,COMPLETED_EFFECTIVE,11,24
3,0.85,0.993,False,COMPLETED_INEFFECTIVE,4,24
4,1.00,0.993,True,COMPLETED_EFFECTIVE,5,24
5,0.94,0.993,False,COMPLETED_INEFFECTIVE,4,24
